# Praca domowa 1

### Przemysław Olender

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
fires = pd.read_csv('forest_fires_dataset.csv')

fires.head()

Analizujemy zbiór danych o pożarach w północnej Portugalii. Według opisu autora cała ramka służy po przewidywania powierzchni pożary za pomocą pozostałych zminnych, są to:

 X - x-owa współrzedna wewnątrz parku
 
 Y - y-owa współrzędna wewnątrz parku

month - miesiąc

day - dzień tygodnia

FFMC - wskaźnik wilgotności paliw* ze ściółki leśnej

DMC - wskaźnik wilgotności paliw* znajdujących się poniżej ściółki leśnej

DC - wskaźnik wilgotności paliw* wgłąb gleby

ISI - wskażnik łączocy wilgotność martwych paliw* i prędkości wiatru, pomaga w oszacowaniu prędkości roznoszenia sie pożaru

temp - temperatura w stopniach celsiujsza

RH - względna wilgotność powietrza wyrażona w %

wind - prędkość wiatru w km/godzinę

rain - wielkość opadów w mm/$m^2$

area - spalona powierzchnia w hektarach

FFMc, DMC, DC, ISI to indeks z międzynaroowego systemu FWI (Fire Weather Index) określającego zagrożenie pożarowe (informazje ze strony https://www.nwcg.gov/publications/pms437/cffdrs/fire-weather-index-system). 

\* - paliwo oznacza materiał podatny na spalenie, w tym przyapdku rośliny

Źródło danych: https://www.apispreadsheets.com/datasets/129


In [ ]:
fires.info()

W danych nie ma żadnych braków, nie będziemy musieli się poźniej martwić wypełniem dziur.

In [ ]:
fires.describe()

In [ ]:
fires.hist(bins = 20, figsize=(18, 12))

plt.show()

Jak widać na wykresach, temperatura ma rozkład bliski normalnemu, wskaźniki FWI i RH mają rozkłady skośne.
Prawie wszytskie wartości w kolumnie area są badrzo bliskie zero, użyjmy więc skali logarytmicznej, żeby lepiej przyjrzeć się wykresowi.

In [ ]:
sns.displot(np.log1p(fires['area']))
plt.show()

Sprawdżmy ile wartości jest równych 0:

In [ ]:
sum(fires['area'] == 0)

Aż 247 z 517 wartości w kolumnie area to 0, może  to oznaczać błąd w danych lub to, że większość pożarów w zaookrągleniu nie stawiło nawet jednego hektara lasu.

Sprawdźmy jakie korelacje zachodzą miedzy danymi:

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(fires.corr(), annot=True, square=True)
#fig = plt.figure(fizsize= (10, 10))
plt.show()

Największe korelacje zachodzą między wskaźnikami FWI i temperaturę, nie jest to duże zaskoczenie, ponieważ im wyższa temperatura tym rośliny są suchsze. Podobnej korelacji można było sie spodziewać między wielkościa opadów (kolumna rain) a wkaźnikamim jednak jak widzieliśmy wcześniej opady prawie nie występują w badanym czasie. Sprawdźmy jak wyglądają scatterploty dla tych zmiennych:

In [ ]:
cols = ['FFMC', 'DMC', 'DC', 'ISI', 'temp']

sns.pairplot(fires, y_vars=fires[cols], x_vars=fires[cols])
plt.show()

In [ ]:
sns.pairplot?

Na wykresach ze zminnymi ISI i FFMC można zauważyć odstające wartość, usuńmy je, żeby nie zaburzały wykresów.

In [ ]:
tmp = fires[fires['ISI'] < 50 ]
tmp = tmp[tmp['FFMC'] > 60]


sns.pairplot(tmp, y_vars=fires[cols], x_vars=fires[cols])
plt.show()

Przyjzyjmy się jeszcze zależnościom zmiennej area od innych zmiennych numerycznych, użyjemu loarytmicznego przekształcenia powirzchni.:

In [ ]:
fires_log = fires
fires_log['area'] = np.log1p(fires['area'])

sns.pairplot(fires_log, y_vars="area", x_vars=fires.columns.values)
plt.show()

Sprawdźmy jak pożary o danej powierzchni rozkładaja się w poszczególnych dniach tygodnia i miesiącach.  Zoabczmy też, ile pożarów występuje w danych dnia i miesiącach.

In [ ]:
day_order = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
month_order = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']


fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(13, 10))
sns.countplot(data=fires, x='day', order = day_order, color = 'dodgerblue', ax = ax1)
sns.boxplot(data=fires_log, x='day', y = 'area', order = day_order, color = 'dodgerblue', ax = ax2)
sns.countplot(data=fires, x='month', order = month_order, color = 'dodgerblue', ax = ax3)
sns.boxplot(data=fires_log, x='month', y= 'area', order = month_order, color = 'dodgerblue', ax = ax4)

ax1.set_title('Liczba pożarów w poszczególne dni tygodnia')
ax2.set_title('Rozkład powierzchni pożarów w poszczególne dni tygodnia')
ax3.set_title('Liczba pożarów w poszczególnych miesiącach')
ax4.set_title('Rozkład powierzchni pożarów w poszczególnych miesiącach')

plt.show()

Liczba pożarów jest największa w weekeny, może się to wiązać z działalnością człowieka i większą liczbą odwiedzających park w weekend.

Jeśli chodzi o rozłożenie pożarów w ciągu roku, to w lipicu i sierpieniu liczba pożarów znacznie większa niż w pozostałych miesiącach. Sprawdźmy jak wygląda rozłożenie temperatury i wilgotności w ciągu roku, żeby ocenić powiązanie tych cech z liczbą pożarów.

Jenak i w ciągu poszczególnych dni jak i miesięcy powierzchnia pożarów utrzymuje się na podobnym poziomie.

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(13, 10))
sns.boxplot(data = fires, x = 'month', y = 'temp', order = month_order, ax = ax1, color = 'white')
sns.boxplot(data = fires, x = 'month', y = 'RH', order = month_order, ax = ax2, color = 'white')
sns.boxplot(data = fires, x = 'month', y = 'wind', order = month_order, ax = ax3, color = 'white')
sns.boxplot(data = fires, x = 'month', y = 'rain', order = month_order, ax = ax4, color = 'white')

ax1.set_title('Rozkład temperatury w ciągu roku')
ax2.set_title('Rozkład wilgotności w ciągu roku')
ax3.set_title('Rozkład prędkości wiatru w ciągu roku')
ax4.set_title('Rozkład opadów wiatru w ciągu roku')
plt.show()

Jak widać temperatura w lipcu i sierpniu jest najwyższa w ciągu rokum jednak nie jest dużo większa niż w czerwcu i wrześniu. Wilgotność również znacząco nie odstaje od innych miesięcy letnich.

Widać korelację dużej prekości wiatru w grudniu i dużej powierzchni strawionej pożarem (z poprzedniego wykresu), nawet przy małej liczbie pożarów (mimo, że pożarów było mało to każdy był stosunkowo duży).

Sprawdźmy jeszcze rozłożenie wskaźników FWI w ciągu roku.

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(13, 10))
sns.boxplot(data = fires, x = 'month', y = 'FFMC', order = month_order, ax = ax1, color = 'white')
sns.boxplot(data = fires, x = 'month', y = 'DMC', order = month_order, ax = ax2, color = 'white')
sns.boxplot(data = fires, x = 'month', y = 'DC', order = month_order, ax = ax3, color = 'white')
sns.boxplot(data = fires, x = 'month', y = 'ISI', order = month_order, ax = ax4, color = 'white')

ax1.set_title('Rozkład wskaźnika FFMC w ciągu roku')
ax2.set_title('Rozkład wskaźnika DMC w ciągu roku')
ax3.set_title('Rozkład wskaźnika DC w ciągu roku')
ax4.set_title('Rozkład wskaźnika ISI w ciągu roku')

plt.show()